# Assignment 3

Author: [Lucas David](http://github.com/lucasdavid)  
This notebook can be downloaded at https://github.com/lucasdavid/mo850/

In [31]:
import os
from itertools import combinations

import numpy as np
import pandas as pd
import scipy
from scipy import stats
import scikits.bootstrap as bootstrap
import statsmodels.stats.contingency_tables
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests

import matplotlib
from matplotlib import pyplot

%matplotlib inline

## Unpaired Data

In [67]:
data_dir = '../data/3/'

datasets = [letter + '.csv' for letter in 'abcde']
print('The following datasets will be loaded:', *datasets)

datasets = [pd.read_csv(os.path.join(data_dir, d), header=None, names=['measure'])
            for d in datasets]

for index, d in enumerate(datasets):
    d['group'] = index

print('Sample of a loaded dataset:',
      datasets[0].head(),
      sep='\n')

The following datasets will be loaded: a.csv b.csv c.csv d.csv e.csv
Sample of a loaded dataset:
    measure  group
0  4.249030      0
1  5.542826      0
2  5.161981      0
3  2.267553      0
4  4.155343      0


In [42]:
s, p = stats.f_oneway(*(d['measure'] for d in datasets))
print('p-value for anova test:', p)

p-value for anova test: 3.3595478270572274e-12


In [43]:
merged_datasets = pd.concat(datasets)
r = pairwise_tukeyhsd(merged_datasets['measure'],
                      groups=merged_datasets['group'])
print(r)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  0      1    -0.5373   -1.514 0.4395 False 
  0      2     1.831    0.8718 2.7901  True 
  0      3     1.908    1.0037 2.8123  True 
  0      4     1.7006   0.7414 2.6597  True 
  1      2     2.3682   1.3405 3.396   True 
  1      3     2.4453   1.4685 3.422   True 
  1      4     2.2379   1.2101 3.2656  True 
  2      3     0.077   -0.8821 1.0362 False 
  2      4    -0.1304  -1.1414 0.8807 False 
  3      4    -0.2074  -1.1665 0.7518 False 
--------------------------------------------


In [44]:
s, p = stats.kruskal(*(d['measure'] for d in datasets))
print('p-value for Kruskal-Wallis:', p)

p-value for Kruskal-Wallis: 2.275373540934311e-09


In [60]:
pairs_indices = list(combinations('abcde', 2))
pairs_data = combinations([d['measure'] for d in datasets], 2)

ps = [stats.ranksums(x, y)[1] for x, y in pairs_data]
ps_holm = multipletests(ps, method='holm')[1]
ps_bonferroni = multipletests(ps, method='bonferroni')[1]

ps = pd.DataFrame({
    'pairs': pairs_indices,
    'p': ps,
    'p holm': ps_holm,
    'p bonferroni': ps_bonferroni
}).set_index('pairs')[['p', 'p holm', 'p bonferroni']]


print(ps)

                   p    p holm  p bonferroni
pairs                                       
(a, b)  2.433450e-01  0.973380      1.000000
(a, c)  1.332963e-04  0.000800      0.001333
(a, d)  5.508804e-06  0.000050      0.000055
(a, e)  1.515892e-04  0.000800      0.001516
(b, c)  6.598469e-06  0.000053      0.000066
(b, d)  9.583666e-07  0.000010      0.000010
(b, e)  2.789325e-05  0.000195      0.000279
(c, d)  5.452595e-01  1.000000      1.000000
(c, e)  8.505281e-01  1.000000      1.000000
(d, e)  5.883647e-01  1.000000      1.000000


## Paired Data

In [75]:
dataset = 'multi.csv'
print('The following datasets will be loaded:', dataset)

dataset = pd.read_csv(os.path.join(data_dir, dataset), header=None)
print(dataset.head())

The following datasets will be loaded: multi.csv
           0          1          2           3           4
0  34.381581  38.230745  52.236630   59.027814   28.288420
1  78.475309  74.647168  82.374582   95.970556   28.983943
2   5.676301   8.919621  16.492051   28.646045   29.585645
3  90.357392  90.869337  98.987833  112.835174  118.070135
4  72.198253  71.068576  79.561990   92.640627   72.266390


In [80]:
s, p = stats.friedmanchisquare(*(dataset[c] for c in dataset.columns))
print('p-value:', p)

p-value: 9.195616602651903e-09
